In [1]:
import sys
sys.path.insert(0,'C:\home\distributed_processing\python')

In [2]:
import random

from distributed_processing.serializers import DummySerializer
from distributed_processing.client import Client
from distributed_processing.filesystem_connector import FileSystemConnector


In [3]:
DIRECTORY = 'G:\home\prueba_ditribuida_fs'
NAMESPACE = "tasks"

In [4]:
import logging
logging.basicConfig(level=logging.DEBUG)

In [5]:
fs_connector = FileSystemConnector(directory=DIRECTORY, namespace=NAMESPACE)

client = Client(DummySerializer(), fs_connector, check_registry="cache")

Client with id: redis_client_2
Results queue: redis_client_2_responses


In [6]:
fs_connector.methods_registry()

{'add': ['requests_cola_1'],
 'mul': ['requests_cola_1'],
 'div': ['requests_cola_1'],
 'eval_py_function': ['requests_py_eval']}

In [7]:
y = client.rpc_async("add", [1, 0])

In [8]:
y.get()

1

In [9]:
client.check_registry ="always"

try:
    y = client.rpc_async("fake", [1, 0])
except Exception as e:
    print(e)

'method_fake'


In [10]:
client.check_registry ="never"
client.set_default_queue("cola_1")

y = client.rpc_async("fake", [1, 0])

try:
    y.get()
except Exception as e:
    print(e)


Error -32601 : The method does not exist/is not available.

 


In [11]:
client.check_registry ="never"
client.set_default_queue("cola_1")

y = client.rpc_async("fake", [1, 0])

y.safe_get(default="Esto es un error")

'Esto es un error'

In [12]:
client.check_registry ="cache"

In [13]:
x = client.rpc_async("div", [1, 0])

In [14]:
try:
    x.get()
except Exception as e:
    print(e)

Error -32603 : Internal RPC error.

 Traceback (most recent call last):
  File "C:\home\distributed_processing\python\distributed_processing\worker.py", line 152, in _process_single_request
    result = func[request["method"]](*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\agarcia\AppData\Local\Temp\ipykernel_28748\1866336200.py", line 8, in div
    return x / y
           ~~^~~
ZeroDivisionError: division by zero



In [15]:
client.rpc_sync("add", [1, 1])

2

In [16]:
def f(x,y): return x + y

y = client.rpc_async_fn(f, [1, 2.0])

In [17]:
y.get()

3.0

In [18]:
client.rpc_sync_fn(f, [3.0, 2.0])

5.0

In [19]:
fs =[]
tp = []
N = 10
for i in range(N):
    fn = random.choice(("add", "mul", "div"))
    t = (fn, [random.random(),random.random()], {})
    print(t)
    tp.append(t)
    fs.append(client.rpc_async(t[0], t[1]))

('mul', [0.369688217971215, 0.8599820764337363], {})
('div', [0.15232705822894432, 0.6588022745370651], {})
('add', [0.6779729829396444, 0.7990150426514161], {})
('mul', [0.4768787023477128, 0.2518003268176241], {})
('add', [0.9557533612088991, 0.2561587541042152], {})
('add', [0.8607958443778821, 0.2719423254847959], {})
('div', [0.32757078956262076, 0.710042099339389], {})
('add', [0.3429297435266645, 0.28981122161407114], {})
('div', [0.320243883649999, 0.9797169243725438], {})
('add', [0.08475659637039368, 0.32498440665329753], {})


In [20]:
[f.get() for f in fs]

[0.31792524132397315,
 0.2312181729123253,
 1.4769880255910603,
 0.12007821310351857,
 1.2119121153131143,
 1.132738169862678,
 0.46133995416241796,
 0.6327409651407356,
 0.3268738915121815,
 0.4097410030236912]

In [21]:
tp

[('mul', [0.369688217971215, 0.8599820764337363], {}),
 ('div', [0.15232705822894432, 0.6588022745370651], {}),
 ('add', [0.6779729829396444, 0.7990150426514161], {}),
 ('mul', [0.4768787023477128, 0.2518003268176241], {}),
 ('add', [0.9557533612088991, 0.2561587541042152], {}),
 ('add', [0.8607958443778821, 0.2719423254847959], {}),
 ('div', [0.32757078956262076, 0.710042099339389], {}),
 ('add', [0.3429297435266645, 0.28981122161407114], {}),
 ('div', [0.320243883649999, 0.9797169243725438], {}),
 ('add', [0.08475659637039368, 0.32498440665329753], {})]

In [22]:
fs = client.rpc_batch_async(tp)

In [23]:
[f.get() for f in fs]

[0.31792524132397315,
 0.2312181729123253,
 1.4769880255910603,
 0.12007821310351857,
 1.2119121153131143,
 1.132738169862678,
 0.46133995416241796,
 0.6327409651407356,
 0.3268738915121815,
 0.4097410030236912]

In [24]:
client.rpc_batch_sync(tp)

[0.31792524132397315,
 0.2312181729123253,
 1.4769880255910603,
 0.12007821310351857,
 1.2119121153131143,
 1.132738169862678,
 0.46133995416241796,
 0.6327409651407356,
 0.3268738915121815,
 0.4097410030236912]

In [25]:
fs =[]
tp = []
N = 10
for i in range(N):
    fn = random.choice(("add", "mul", "div", "fake"))
    t = (fn, [random.random(),random.random()], {})
    print(t)
    tp.append(t)

('fake', [0.7923043905922567, 0.2682549812963724], {})
('add', [0.6989850212795357, 0.752227485322533], {})
('div', [0.6117717750210461, 0.09287398583118278], {})
('add', [0.020927123973833672, 0.3123942429279394], {})
('add', [0.9608606191132902, 0.8766762990276146], {})
('fake', [0.2903476244590122, 0.119907307035243], {})
('mul', [0.8819358297967668, 0.7821960650188758], {})
('div', [0.9159494118127299, 0.8944396627782636], {})
('fake', [0.984324424926479, 0.03654655317328659], {})
('fake', [0.7608252322809145, 0.237856608477526], {})


In [26]:
tp

[('fake', [0.7923043905922567, 0.2682549812963724], {}),
 ('add', [0.6989850212795357, 0.752227485322533], {}),
 ('div', [0.6117717750210461, 0.09287398583118278], {}),
 ('add', [0.020927123973833672, 0.3123942429279394], {}),
 ('add', [0.9608606191132902, 0.8766762990276146], {}),
 ('fake', [0.2903476244590122, 0.119907307035243], {}),
 ('mul', [0.8819358297967668, 0.7821960650188758], {}),
 ('div', [0.9159494118127299, 0.8944396627782636], {}),
 ('fake', [0.984324424926479, 0.03654655317328659], {}),
 ('fake', [0.7608252322809145, 0.237856608477526], {})]

In [27]:
client.check_registry ="never"
client.set_default_queue("cola_1")

fs = client.rpc_batch_async(tp)

In [28]:
[f.safe_get() for f in fs]

[None,
 1.4512125066020687,
 6.5871166133976935,
 0.33332136690177305,
 1.8375369181409047,
 None,
 0.689846735666188,
 1.0240482951836616,
 None,
 None]

In [29]:
client.check_registry ="cache"

In [30]:
try:
    x = client.rpc_batch_sync(tp, timeout=5)
except Exception as e:
    print(e)

Method fake does not exist/is not available.


In [31]:
try:    
    x = client.rpc_batch_sync(tp, timeout=5)
except Exception as e:
    print(e)

Method fake does not exist/is not available.


In [32]:
client.check_registry="never"
client.set_default_queue("cola_1")

x = client.rpc_async("kk", [1, 0])

In [33]:
try:
    x.get()
except Exception as e:
    print(e)

Error -32601 : The method does not exist/is not available.

 


In [34]:
y = client.rpc_async("add", [1, 0])

In [35]:
y.get(5)

1

In [36]:
def f(x,y): return x + y

client.check_registry="never"
y = client.rpc_async_fn(f, [1, 2.0])
try:
    y.get()
except Exception as e:
    print(e)

Error -32601 : The method does not exist/is not available.

 
